In [15]:
import pandas as pd
import numpy as np
from joblib import load
import os

In [16]:
# Load trained model
models_dir = os.path.join("../", "src", "models")
spring_temp_model = load(os.path.join(models_dir, "spring_temp_model.joblib"))
am_model = load(os.path.join(models_dir, "am_transparency_model.joblib"))
pm_model = load(os.path.join(models_dir, "pm_transparency_model.joblib"))
fish_model = load(os.path.join(models_dir, "fish_survial_model.joblib"))
print("✅ All models loaded (with pipelines)")


✅ All models loaded (with pipelines)


In [17]:
# === Step 1: Input data (sample) ===
X_input = pd.DataFrame([{
    "Max air temp": 69,
    "Min air temp": 44,
    "Dec Rain": 0.5,
    "Calmar Rain": 0.5,
    "Season": "Spring",
    "Day of Year": 120,
    "Fish Age": 1.5,
    "Max Air Temp x Rain": 67 * (0.2 + 0.1),
    "Total Rain": 0.2 + 0.1,
    "Dec Rain (Lag 3)": 0.25,
    "Calmar Rain (Lag 3)": 0.15,
    "Max air temp (Lag 3)": 65,
    "Dec Rain 7-day avg": 0.3,
    "Calmar Rain 7-day avg": 0.2,
    "Max air temp 7-day avg": 66,
    "# fish": 30000  

}])

In [18]:
# === Step 2: Predict Spring Temp ===
spring_temp = spring_temp_model.predict(X_input)[0]
print(f"\n🌡️ Predicted Spring Temp (F): {spring_temp:.2f}")

# Add Spring Temp output to input
X_input["Spring Temp (F)"] = spring_temp
X_input["Spring_Temp x Rain"] = spring_temp * (X_input["Dec Rain"] + X_input["Calmar Rain"])
X_input["Spring Temp (F) (Lag 3)"] = spring_temp - 1.0
X_input["Spring Temp (F) 7-day avg"] = spring_temp - 0.3

# === Step 3: Predict AM Transparency ===
am_transparency = am_model.predict(X_input)[0]
print(f"💧 Predicted AM Transparency: {am_transparency:.2f}")
X_input["AM Transparency"] = am_transparency

# === Step 4: Predict PM Transparency ===
pm_transparency = pm_model.predict(X_input)[0]
print(f"💧 Predicted PM Transparency: {pm_transparency:.2f}")
X_input["PM Transparency"] = pm_transparency

# Add all remaining expected fish model features
X_input["AM Feed"] = "X"
X_input["PM Feed"] = "X"
X_input["AM Transparency (Lag 3)"] = am_transparency - 0.5
X_input["PM Transparency (Lag 3)"] = pm_transparency - 0.5
X_input["AM Transparency 7-day avg"] = am_transparency - 0.3
X_input["PM Transparency 7-day avg"] = pm_transparency - 0.3


# === Step 5: Predict Fish Survival ===
fish_survival = fish_model.predict(X_input)[0]
print(f"\n🐟 Predicted Fish Survival Rate: {fish_survival:.2f}")


🌡️ Predicted Spring Temp (F): 49.57
💧 Predicted AM Transparency: 5.57
💧 Predicted PM Transparency: 11.45

🐟 Predicted Fish Survival Rate: 99.95


In [20]:
models_dir = os.path.join("../", "src", "models")
spring_temp_model = load(os.path.join(models_dir, "spring_temp_model.joblib"))

In [21]:
type(spring_temp_model)


sklearn.pipeline.Pipeline

In [24]:
features = spring_temp_model.named_steps["preprocessor"].get_feature_names_out()
print(features)

['num__Max air temp' 'num__Min air temp' 'num__Dec Rain'
 'num__Calmar Rain' 'num__Max Air Temp x Rain' 'num__Total Rain'
 'cat__Season_Fall' 'cat__Season_Spring' 'cat__Season_Summer'
 'cat__Season_Winter']
